# Project - Introduction to Python Programming

The feedback for your project will be written in the Markdown cell under this one, containing "YOUR ANSWER HERE" -- please *do not delete it*! Create new cells *below the rules* to write your project.

# FEEDBACK

---
---

# Introduction to Hierarchical Temporal Memory (HTM)
## Tutorial Sheet 1 - Sparse Distributed Representations (SDRs)
#### Tutorial goals:
1. Understand the motivation for using SDRs
2. Learn the key features of SDRs
3. Learn what an encoder is - 4 simple rules
4. Implement an encoder for stock data, and convert stock data set into SDRs

#### Motivations

One of the most interesting challenges in AI is the problem of knowledge representation. Representing everyday facts and relationships in a form that computers can work with has proven to be difficult with traditional computer science methods. The  basic problem is that our knowledge of the world  is not divided into discrete facts with well-defined relationships. Almost everything we know  has exceptions, and the relationships between concepts are too numerous and ill-defined to map onto traditional computer data structures. 

Brains do not have this problem. They represent information using a method called Sparse Distributed Representations, or SDRs. SDRs and their mathematical properties are essential for biological intelligence. Everything the brain does and every principle described in this book is based on SDRs. SDRs are the language of the brain. The flexibility and creativity of human intelligence is inseparable from this representation method. Therefore, if we want intelligent machines to be similarly flexible and creative,they need to be based on the same representation method, SDRs. 

An SDR consists of thousands of bits where at any point in time a small percentage of the bits are 1’s and the rest are 0’s. The bits in an SDR correspond to neurons in the brain, a 1 being a relatively active neuron and a 0 being a relatively inactive neuron. The most important property of SDRs is that each bit has meaning. Therefore, the set of active bits in any  particular representation encodes the set of semantic attributes of what is being represented. The  bits are not labeled (that is to say, no one assigns meanings to the bits), but rather, the semantic meanings of bits are learned. If two  SDRs have active bits in  the same locations, they share the semantic attributes represented by those bits. By determining the overlap between two SDRs(the equivalent bits that are 1 in both SDRs) we can immediately see how two representations are semantically similar and how they are semantically different. Because of this semantic overlap property, systems based on SDRs automatically generalize based on semantic similarity.[[0]][Hawkins, J. et al. 2016. Biological and Machine Intelligence.]

[Hawkins, J. et al. 2016. Biological and Machine Intelligence.]: https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-SDR.pdf

#### Key features of SDRs

1. Only a small number of neurons (bits) are firing (=1) at any point in time
2. There are a very large number of neurons (bits)
3. Every bit represents something different and has meaning – semantically similar data will have large overlap in their SDR
4. Information is distributed and no single neuron (bit) is critical
5. SDRs enable extremely fast computation (All computations run linearly with the number of ON bits)

#### Some further features to ponder...
Think about why SDRs might be robust to noise. Why might it not matter if an SDR had a few bits missing/in a random position. Could you still tell what it was supposed to match? If it was matched wrongly, what would it likely be matched to?

If you were given a random sample of the on bits from an sdr, could you tell which sdr this was from a list of many?

#### What is an Encoder?

An encoder simply transforms some form of data into SDRs.

There are a few rules to follow when writing an encoder:

1. Semantically similar input data should result in SDRs with overlapping active bits.
2. The same input should always produce the same SDR as output.
3. The output should have the same dimensionality (total number of bits) for all inputs.
4. The  output should have similar sparsity for all inputs and have enough one-bits to handle noise and subsampling.[[2]][Hawkins, J. et al. 2016. Biological and Machine Intelligence.]

[Hawkins, J. et al. 2016. Biological and Machine Intelligence.]: https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

### Implementing an encoder

Now it's time to implement an encoder for stock data

In [7]:
import pandas as pd
import numpy as np
import datetime

#### Download the data, and read it into a panda dataframe

The data I worked with I found here: https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/audcad/2007

If you prefer to work on some other sequential data: feel free!

The following code should be edited to include the file path where you decide to store the data in csv format.

In [5]:
# Read our csv into a panda dataframe
data = pd.read_csv("YOUR/FILE/PATH.csv")
# Give columns names
data = data.rename(	dict(zip(list(data.columns.values),
            ['datetime','a','b','c','d','e'])),
            errors="raise", 
            axis=1)
# Convert datetime column to proper datetime object Series
data['datetime'] = pd.to_datetime(data['datetime'])
# Print the head as a quick check
print(data.head())

FileNotFoundError: [Errno 2] File b'YOUR/FILE/PATH.csv' does not exist: b'YOUR/FILE/PATH.csv'

#### Encoding functions

In [10]:
def datetime_to_sdr(date_time, minimum, maximum):
    """
    Takes datetime.datetime object and returns sdr.
    
    This function takes a datetime.datetime object, and returns a sdr consisting of 4 parts:
    1. a representation of the day of the week
    2. a representation of the time of year/season
    3. a representation of the year
    4. a representation of the time of day
    It does this given a upper and lower bound on the dates that can be expressed
    
    Parameters
    ----------
    date_time : datetime.datetime
        The datetime.datetime to be express
    minimum : datetime.datetime
        The lower bound for datetimes that can be expressed
    maximum : datetime.datetime
        The upper bound for datetimes that can be expressed
    
    Returns
    -------
    list
        A list sdr for the datetime given - expressed with 0s and 1s.
    """
    
    # Define the number of active bits for each part of representation
    # We hold this constant so that no part of the sdr is weighted over
    # any other.
    active_n = 10

    day_sdr = datetime_to_day_sdr(date_time, active_n)
    season_sdr = datetime_to_season_sdr(date_time, active_n)
    year_sdr = datetime_to_year_sdr(date_time, minimum, maximum, active_n)
    time_sdr = datetime_to_time_sdr(date_time, active_n)

    date_time_sdr = day_sdr+season_sdr+year_sdr+time_sdr

    return date_time_sdr

def datetime_to_day_sdr(date_time, active_n):
    """
    Takes datetime.datetime object and returns sdr representing the day of the week.

    Parameters
    ----------
    date_time : datetime.datetime
        The datetime.datetime to be express
    active_n : int
        The number of active bits that the representation should contain.
    
    Returns
    -------
    list
        A list sdr for the day of the week - expressed with 0s and 1s.
    """
    day = date_time.weekday()
    day_n = 7*active_n
    day_sdr = [0]*(day*active_n) + [1]*active_n
    day_sdr = day_sdr + [0]*(day_n-len(day_sdr))
    return day_sdr

def datetime_to_season_sdr(date_time, active_n):
    """
    Takes datetime.datetime object and returns sdr representing the time of year/season.

    Takes datetime.datetime object and returns sdr representing the time of year/season with resolution
    to the day.

    Parameters
    ----------
    date_time : datetime.datetime
        The datetime.datetime to be express
    active_n : int
        The number of active bits that the representation should contain.
    
    Returns
    -------
    list
        A list sdr for the time of year/season - expressed with 0s and 1s.
    """
    season = date_time.timetuple().tm_yday-1
    season_n = 366+(active_n-1)
    season_sdr = [0]*season + [1]*active_n+[0]*(season_n-season-active_n)
    return season_sdr

def datetime_to_year_sdr(date_time, minimum, maximum, active_n):
    """
    Takes datetime.datetime object and returns sdr.
    
    This function takes a datetime.datetime object, and returns a sdr for the year.
    It does this given a upper and lower bound on the dates that can be expressed.
    
    Parameters
    ----------
    date_time : datetime.datetime
        The datetime.datetime to be express
    minimum : datetime.datetime
        The lower bound for datetimes that can be expressed
    maximum : datetime.datetime
        The upper bound for datetimes that can be expressed
    active_n : int
        The number of active bits that the representation should contain.
    
    Returns
    -------
    list
        A list sdr for the year - expressed with 0s and 1s.
    """
    year_n = (maximum.year-minimum.year+1)+(active_n-1)
    year = date_time.year-minimum.year
    year_sdr = [0]*year + [1]*active_n+[0]*(year_n-year-active_n)
    return year_sdr

def datetime_to_time_sdr(date_time,active_n):
    """
    Takes datetime.datetime object and returns sdr representing the time of day.

    Takes datetime.datetime object and returns sdr representing the time of day with resolution 
    to the second.

    Parameters
    ----------
    date_time : datetime.datetime
        The datetime.datetime to be express
    active_n : int
        The number of active bits that the representation should contain.
    
    Returns
    -------
    list
        A list sdr for the time of day - expressed with 0s and 1s.
    """
    time_n = 24*60*60+(active_n-1)
    time = date_time.time()
    seconds_since_midnight = int((date_time - 
                                  datetime.datetime(date_time.year, date_time.month, date_time.day)).total_seconds())
    time_sdr = [0]*seconds_since_midnight + [1]*active_n + [0]*(time_n-seconds_since_midnight-active_n)
    return time_sdr	

def stock_price_to_sdr(price, minimum=0., maximum=100., resolution=0.0001):
    """
    Takes stock_price and returns a suitable sdr.

    Takes a float for the stock price. It then represents this this price as an sdr with capacity defined
    by the minimum and maximum parameters and with a resolution given by the resolution parameter.

    Parameters
    ----------
    price : float
        The value to be expressed
    minimum : float
        The smallest value that can be expressed
    maximum : float
        The largest value that can be expressed
    
    Returns
    -------
    list
        A list sdr for the stock price - expressed with 0s and 1s.
    """
    
    assert price>=minimum,"Price is smaller than minimum"
    assert price<=maximum,"Price is larger than maximum"

    price_i = x_to_res_i(price, resolution)
    n_sdr = int(np.ceil((maximum-minimum)/resolution))
    # Portion of active bits should be ~5%
    active = int(np.ceil(n_sdr*.05))
    # Bits added to beginning and end to avoid cyclic definition
    n_sdr+=int(np.ceil(active/2)*2)

    sdr = [0]*price_i+[1]*active
    sdr = sdr + [0]*(n_sdr-len(sdr))
    return sdr
    
def stock_price_to_sdr_test():
    # Test that sdr is always same size
    return False

def x_to_res_i(x,res):
    """
    Takes a float value x and returns its nearest multiple of the float value res, divided by res.

    Parameters
    ----------
    x : float
        The value to be expressed
    res : float
        The divisor
    
    Returns
    -------
    int
        The number of times that res fits into the nearest multiple of res to x.
    """
    r=x%res
    if r>res/2:
        m=(x-r)/res+1
    else:
        m=(x-r)/res
    return int(m)

def compress_sdr(sdr):
    """
    Expresses an sdr by a list of indices of the active bits.

    Parameters
    ----------
    sdr : list
        An sdr expressed by 0s and 1s
    
    Returns
    -------
    list
        A list of indices of the active bits in the input sdr.
    """
    return [i for i,n in enumerate(sdr) if n==1]

def set_sdr(sdr):
    """
    Expresses an sdr by a set of indices of the active bits.

    Parameters
    ----------
    sdr : list
        An sdr expressed by 0s and 1s
    
    Returns
    -------
    set
        A set of indices of the active bits in the input sdr.
    """
    return set(compress_sdr(sdr))

#### Converting our full dataset to sdr representations

Note that this should be edited to inlude your output file path of choice.

In [ ]:
# Get the datetime Series, and find its range
date_time = data['datetime']
minimum_date = date_time.min()
maximum_date = date_time.max()

# Get the first of the price columns (Stock data has multiple dimensions for each time step.
# We ignore there specific meanings for these purposes
prices = data['a']

# We map the date_time and price entries to sdrs
# This may take a while! Go get yourself a coffee.
date_time = date_time.map(lambda x: set_sdr(datetime_to_sdr(x, minimum_date, maximum_date)))
prices = prices.map(lambda x: set_sdr(stock_price_to_sdr(x, minimum=prices.min(), maximum=prices.max())))

# Write the series to a new dataframe, and stick it in a csv file somewhere for later
sdr_data = pd.DataFrame({'datetime':date_time, 'price':prices})
sdr_data.to_csv("YOUR/FILE/PATH/sdr_data.csv")

# Print the head as a quick check
print(sdr_data.head())

### Exercises:
1. Rewrite at least one of the encoding functions to return directly the set representation given by set_sdr without going via the 0s and 1s list representation. Why might this be advantages?
2. Write a function that, given a collection of n sdrs, and an sdr that is equal to one of those n, can match the sdr given to one in the set. Use subsampling, and test the how fast and reliable you can make it. Add noise and see if you can still achieve good results.
3. Take n sdrs. Take r<<n randomly chosen sdrs from these n and take their union. Can you tell which sdrs went into the union?